In [1]:
#インポート
from bs4 import BeautifulSoup as bs
import re
import time
import os
import sys
import json
import pandas as pd
import numpy as np
import pickle
import unicodedata

data = pd.read_csv('ufo2024/ufo-2024/table_retrieval/data/dataframe/data.csv')

#### 目次データの作成

In [2]:
def edit_text(text):
    return re.sub(r'[^\u3040-\u309F\u30A0-\u30FF\uFF65-\uFF9F\u4E00-\u9FFF]', '', text)

In [3]:
import pickle
def make_contents(doc_id):
    try:
        with open(f'ntcir18-u4/table_retrieval/data/contents/{doc_id}.pkl', 'rb') as f:
            dct = pickle.load(f)
            f.close()
        return dct
    except:
        dct = pd.DataFrame()
        files = os.listdir(f'ntcir18-u4/table_retrieval/test_tr/reports_tr_test/{doc_id}')
        tmp_h1 = ''
        tmp_h2 = ''
        tmp_h3 = ''
        tmp_h4 = ''
        tmp_h5 = ''
        tmp_h6 = ''
        for file in files:
            with open(f'ntcir18-u4/table_retrieval/test_tr/reports_tr_test/{doc_id}/{file}', 'r') as f:
                soup = bs(f, 'html.parser')
                f.close()
            html_tag = soup.find_all(['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'table'])
            for index, tag in enumerate(html_tag):
                if tag.name == 'h1':
                    text = unicodedata.normalize('NFKC', tag.get_text()).replace('\n', '').replace(' ', '')
                    if text == '':
                        continue
                    tmp_h1 = edit_text(text)
                    tmp_h2 = ''
                    tmp_h3 = ''
                    tmp_h4 = ''
                    tmp_h5 = ''
                    tmp_h6 = ''
                elif tag.name == 'h2':
                    text = unicodedata.normalize('NFKC', tag.get_text()).replace('\n', '').replace(' ', '')
                    if text == '':
                        continue
                    tmp_h2 = edit_text(text)
                    tmp_h3 = ''
                    tmp_h4 = ''
                    tmp_h5 = ''
                    tmp_h6 = ''
                elif tag.name == 'h3':
                    text = unicodedata.normalize('NFKC', tag.get_text()).replace('\n', '').replace(' ', '')
                    if text == '':
                        continue
                    tmp_h3 = edit_text(text)
                    tmp_h4 = ''
                    tmp_h5 = ''
                    tmp_h6 = ''
                elif tag.name == 'h4':
                    text = unicodedata.normalize('NFKC', tag.get_text()).replace('\n', '').replace(' ', '')
                    if text == '':
                        continue
                    tmp_h4 = edit_text(text)
                    tmp_h5 = ''
                    tmp_h6 = ''
                elif tag.name == 'h5':
                    text = unicodedata.normalize('NFKC', tag.get_text()).replace('\n', '').replace(' ', '')
                    if text == '':
                        continue
                    tmp_h5 = edit_text(text)
                    tmp_h6 = ''
                elif tag.name == 'h6':
                    text = unicodedata.normalize('NFKC', tag.get_text()).replace('\n', '').replace(' ', '')
                    if text == '':
                        continue
                    tmp_h6 = edit_text(text)
                elif tag.name == 'table':
                    dct =pd.concat([dct, pd.DataFrame({'h1': [tmp_h1], 'h2': [tmp_h2], 'h3': [tmp_h3], 'h4': [tmp_h4], 'h5': [tmp_h5], 'h6': [tmp_h6], 'table_id':[tag.get('table-id')],'html': [file]})])
        pickle.dump(dct, open(f'ntcir18-u4/table_retrieval/data/contents/{doc_id}.pkl', 'wb'))
        return dct

#### テーブル検索

In [18]:
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

api_version="2023-12-01-preview"
engine = "gpt-35" #GPT3.5の場合
 
client = AzureOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),  
    api_version=api_version,
    azure_endpoint = os.getenv("OPENAI_API_BASE")
    )

def extract_noun(query):
    prompt =f"""
    以下の文章から名詞をすべて抽出してください。

    sentence:2020年度時点のRow1Memberにおける「決議日付、剰余金の配当」は?'
    noun:Row1Member
    noun:決議日付
    noun:剰余金
    noun:配当

    sentence:2019年度における「人件費、販売費及び一般管理費」は?
    noun:人件費
    noun:販売費
    noun:一般管理費

    sentence:{query}
    noun:
    """

    response = client.chat.completions.create(
        model= engine,
        messages=[
            {"role": "user", "content": prompt}, 
        ],
        temperature=0,
        max_tokens=2048,
        top_p=0.1,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
    nouns = response.choices[0].message.content
    nouns = nouns.replace(' ', '').split('\n')
    print(nouns)
    return [noun.split(':')[1] for noun in nouns if noun != '']

In [7]:
def search_table(dct, query):
    if len(dct) == 1:
        return {dct['table_id'].values[0]: 1}
    
    words = extract_noun(query)
    table_dct = dict(zip(dct['table_id'].values, np.zeros(len(dct))))
    for index, row in dct.iterrows():
        try:
            with open(f'ntcir18-u4/table_retrieval/data/table/{row["table_id"]}.pkl', 'rb') as f:
                text = pickle.load(f)
                f.close()
        except:
            dct = pd.DataFrame()
            with open(f'ntcir18-u4/table_retrieval/test_tr/reports_tr_test/{row["table_id"].split("-")[0]}/{row["html"]}', 'r') as f:
                soup = bs(f, 'html.parser')
                f.close()
            html_tag = soup.find_all(['table',])
            for index, tag in enumerate(html_tag):
                if tag.get('table-id') == row['table_id']:
                    text = unicodedata.normalize('NFKC', tag.get_text()).replace('\n', '').replace(' ', '')
                    break
            pickle.dump(text , open(f'ntcir18-u4/table_retrieval/data/table/{row["table_id"]}.pkl', 'wb'))
        for word in words:
            table_dct[row['table_id']] += text.count(word)
    return table_dct

#### 検索モデル

In [5]:
'''dct = pd.DataFrame()
tmp = data_tr.copy()
doc_ls = data_tr.drop_duplicates(subset=['doc_id'],)['doc_id'].values
for doc_id in doc_ls:
    dct = pd.concat([dct, make_contents(doc_id)], axis = 0)
    print(doc_id)
tmp = pd.merge(tmp, dct, on='table_id', how='left').dropna()
tmp.to_csv('ntcir18-u4/table_retrieval/data/train_overview.csv', index=False)'''
tmp = pd.read_csv('ntcir18-u4/table_retrieval/data/train_overview.csv')

In [13]:
with open(f'ntcir18-u4/table_retrieval/data/contents/S100INDK.pkl', 'rb') as f:
    dct = pickle.load(f)
    f.close()

In [16]:
dct[20:].head(5)

,h1,h2,h3,h4,h5,h6,table_id,html
0,第一部企業情報,第企業の概況,従業員の状況,連結会社の状況,,,S100INDK-0101010-tab21,0101010.html
0,第一部企業情報,第企業の概況,従業員の状況,提出会社の状況,,,S100INDK-0101010-tab22,0101010.html
0,第一部企業情報,第事業の状況,経営者による財政状態経営成績及びキャッシュ・フローの状況の分析,販売の状況,,,S100INDK-0102010-tab23,0102010.html
0,第一部企業情報,第事業の状況,経営者による財政状態経営成績及びキャッシュ・フローの状況の分析,財政状態の状況,,,S100INDK-0102010-tab24,0102010.html
0,第一部企業情報,第設備の状況,設備投資等の概要,,,,S100INDK-0103010-tab25,0103010.html


In [19]:
count = 0
answer_ls = []
for index, rows in data_te[5132:].iterrows():
    dct = make_contents(rows['doc_id'])
    df = tmp.query('ja_question== @rows["ja_question"]')
    df = df[['h1','h2','h3','h4','h5','h6']].drop_duplicates()
    headers = ['h2','h3','h4','h5']
    for header in headers:
        length = len(dct[dct[header].isin(df[header])])
        if length == 0:
            break
        else:
            dct = dct[dct[header].isin(df[header])]
    table_dct = search_table(dct, rows['question'])
    max_value = max(table_dct.values())
    max_keys = [key for key, value in table_dct.items() if value == max_value]

    answer_ls.append(max_keys[0])

['noun:株式会社オリエンタルランド', 'noun:有形固定資産', 'noun:無形固定資産', 'noun:増加額']
['noun:株式会社オリエンタルランド', 'noun:HotelBusinessReportableSegmentsMember', 'noun:外部顧客', 'noun:売上高']
['noun:株式会社オリエンタルランド', 'noun:HotelBusinessReportableSegmentsMember', 'noun:有形固定資産', 'noun:無形固定資産', 'noun:増加額']
['noun:株式会社オリエンタルランド', 'noun:2020年', 'noun:個別決算', 'noun:NonConsolidatedMember', 'noun:売掛金']
['noun:株式会社オリエンタルランド', 'noun:個別決算', 'noun:NonConsolidatedMember', 'noun:有価証券']
['noun:株式会社オリエンタルランド', 'noun:2020年時点', 'noun:個別決算', 'noun:NonConsolidatedMember', 'noun:商品']
['noun:株式会社オリエンタルランド', 'noun:個別決算', 'noun:NonConsolidatedMember', 'noun:仕掛品']
['noun:株式会社オリエンタルランド', 'noun:2019年時点', 'noun:個別決算', 'noun:NonConsolidatedMember', 'noun:原材料']
['noun:株式会社オリエンタルランド', 'noun:2020年時点', 'noun:個別決算', 'noun:NonConsolidatedMember', 'noun:原材料']
['noun:株式会社オリエンタルランド', 'noun:個別決算', 'noun:NonConsolidatedMember', 'noun:貯蔵品']
['noun:株式会社オリエンタルランド', 'noun:個別決算', 'noun:NonConsolidatedMember', 'noun:前払費用']
['noun:株式会社オリエンタルランド', 'noun:2020年時点', 'nou

In [14]:
ans_tmp = answer_ls

In [21]:
len(data_te)

6474

In [20]:
len(answer_ls)

1342

In [22]:
ans_tmp + answer_ls

['S100IHTB-0101010-tab2',
 'S100IHTB-0101010-tab2',
 'S100IHTB-0101010-tab2',
 'S100IHTB-0101010-tab2',
 'S100IHTB-0101010-tab2',
 'S100IHTB-0101010-tab2',
 'S100IHTB-0101010-tab2',
 'S100IHTB-0101010-tab2',
 'S100IHTB-0101010-tab2',
 'S100IHTB-0101010-tab2',
 'S100IHTB-0101010-tab2',
 'S100IHTB-0101010-tab2',
 'S100IHTB-0101010-tab2',
 'S100IHTB-0101010-tab2',
 'S100IHTB-0101010-tab2',
 'S100IHTB-0101010-tab3',
 'S100IHTB-0101010-tab3',
 'S100IHTB-0101010-tab3',
 'S100IHTB-0101010-tab3',
 'S100IHTB-0101010-tab3',
 'S100IHTB-0101010-tab3',
 'S100IHTB-0101010-tab3',
 'S100IHTB-0101010-tab3',
 'S100IHTB-0000000-tab1',
 'S100IHTB-0101010-tab8',
 'S100IHTB-0101010-tab8',
 'S100IHTB-0101010-tab8',
 'S100IHTB-0104010-tab26',
 'S100IHTB-0104010-tab27',
 'S100IHTB-0104010-tab27',
 'S100IHTB-0104010-tab27',
 'S100IHTB-0104010-tab27',
 'S100IHTB-0104010-tab42',
 'S100IHTB-0104010-tab42',
 'S100IHTB-0104010-tab42',
 'S100IHTB-0104010-tab42',
 'S100IHTB-0104010-tab42',
 'S100IHTB-0104010-tab42',
 

In [16]:
len(ans_tmp)

5132

In [15]:
ans_tmmp

['S100IHTB-0101010-tab2',
 'S100IHTB-0101010-tab2',
 'S100IHTB-0101010-tab2',
 'S100IHTB-0101010-tab2',
 'S100IHTB-0101010-tab2',
 'S100IHTB-0101010-tab2',
 'S100IHTB-0101010-tab2',
 'S100IHTB-0101010-tab2',
 'S100IHTB-0101010-tab2',
 'S100IHTB-0101010-tab2',
 'S100IHTB-0101010-tab2',
 'S100IHTB-0101010-tab2',
 'S100IHTB-0101010-tab2',
 'S100IHTB-0101010-tab2',
 'S100IHTB-0101010-tab2',
 'S100IHTB-0101010-tab3',
 'S100IHTB-0101010-tab3',
 'S100IHTB-0101010-tab3',
 'S100IHTB-0101010-tab3',
 'S100IHTB-0101010-tab3',
 'S100IHTB-0101010-tab3',
 'S100IHTB-0101010-tab3',
 'S100IHTB-0101010-tab3',
 'S100IHTB-0000000-tab1',
 'S100IHTB-0101010-tab8',
 'S100IHTB-0101010-tab8',
 'S100IHTB-0101010-tab8',
 'S100IHTB-0104010-tab26',
 'S100IHTB-0104010-tab27',
 'S100IHTB-0104010-tab27',
 'S100IHTB-0104010-tab27',
 'S100IHTB-0104010-tab27',
 'S100IHTB-0104010-tab42',
 'S100IHTB-0104010-tab42',
 'S100IHTB-0104010-tab42',
 'S100IHTB-0104010-tab42',
 'S100IHTB-0104010-tab42',
 'S100IHTB-0104010-tab42',
 

In [24]:
data_te

,question_number,question,doc_id,ja_question
0,question_test1,積水ハウス株式会社の2018年における「売上高、経営指標等」は？,S100IHTB,2018年における経営指標等の売上高は?
1,question_test2,積水ハウス株式会社の2020年における「売上高、経営指標等」は？,S100IHTB,2020年における経営指標等の売上高は?
2,question_test4,積水ハウス株式会社の2020年における「経常利益又は経常損失（△）、経営指標等」は？,S100IHTB,2020年における経営指標等の経常利益又は経常損失(△)は?
3,question_test7,積水ハウス株式会社の2016年における「包括利益、経営指標等」は？,S100IHTB,2016年における経営指標等の包括利益は?
4,question_test8,積水ハウス株式会社の2018年における「包括利益、経営指標等」は？,S100IHTB,2018年における経営指標等の包括利益は?
...,...,...,...,...
6469,question_test12939,レーザーテック株式会社の2020年時点における「提出日、表紙」は？,S100JRT9,2020年時点における表紙の提出日は?
6470,question_test12941,レーザーテック株式会社の2020年時点における「会社名、表紙」は？,S100JRT9,2020年時点における表紙の会社名は?
6471,question_test12944,レーザーテック株式会社の2020年時点における「本店の所在の場所、表紙」は？,S100JRT9,2020年時点における表紙の本店の所在の場所は?
6472,question_test12945,レーザーテック株式会社の2020年時点における「電話番号、本店の所在の場所、表紙」は？,S100JRT9,2020年時点における表紙の本店の所在の場所の電話番号は?


In [25]:
answer_dct = dict(zip(data_te['question_number'].values, ans_tmp + answer_ls)) 
with open('ntcir18-u4/table_retrieval/test_tr/answersheet_tr_test.json', 'w') as f:
    json.dump(answer_dct, f)

In [26]:
answer_dct

{'question_test1': 'S100IHTB-0101010-tab2',
 'question_test2': 'S100IHTB-0101010-tab2',
 'question_test4': 'S100IHTB-0101010-tab2',
 'question_test7': 'S100IHTB-0101010-tab2',
 'question_test8': 'S100IHTB-0101010-tab2',
 'question_test9': 'S100IHTB-0101010-tab2',
 'question_test11': 'S100IHTB-0101010-tab2',
 'question_test13': 'S100IHTB-0101010-tab2',
 'question_test15': 'S100IHTB-0101010-tab2',
 'question_test16': 'S100IHTB-0101010-tab2',
 'question_test18': 'S100IHTB-0101010-tab2',
 'question_test19': 'S100IHTB-0101010-tab2',
 'question_test21': 'S100IHTB-0101010-tab2',
 'question_test22': 'S100IHTB-0101010-tab2',
 'question_test27': 'S100IHTB-0101010-tab2',
 'question_test31': 'S100IHTB-0101010-tab3',
 'question_test32': 'S100IHTB-0101010-tab3',
 'question_test35': 'S100IHTB-0101010-tab3',
 'question_test38': 'S100IHTB-0101010-tab3',
 'question_test39': 'S100IHTB-0101010-tab3',
 'question_test46': 'S100IHTB-0101010-tab3',
 'question_test47': 'S100IHTB-0101010-tab3',
 'question_test4